##### Clean New Dataset

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
if __name__ == '__main__':
    PATH = '/content/drive/My Drive/DL_Project/notebookks/basit/new_data/'
    DATASET_ENCODING = "ISO-8859-1"
    
    e = ['sadness','fear','confident','analytical','anger']
    files = ['0221_0307', '0308_0323', '0324_0408']
    
    text = []
    timestamp = []
    clean_text = []
    pred_sadness = []
    pred_fear = []
    pred_confident = []
    pred_analytical = []
    pred_anger = []
    
    for f in files:
        
        fname= 'data_test_'+f+'_'+e[0]+'_lite.csv'
        data0 = pd.read_csv(PATH + fname, encoding = DATASET_ENCODING, index_col=0)

        fname= 'data_test_'+f+'_'+e[1]+'_lite.csv'
        data1 = pd.read_csv(PATH + fname, encoding = DATASET_ENCODING, index_col=0)

        fname= 'data_test_'+f+'_'+e[2]+'_lite.csv'
        data2 = pd.read_csv(PATH + fname, encoding = DATASET_ENCODING, index_col=0)

        fname= 'data_test_'+f+'_'+e[3]+'_lite.csv'
        data3 = pd.read_csv(PATH + fname, encoding = DATASET_ENCODING, index_col=0)

        fname= 'data_test_'+f+'_'+e[4]+'_lite.csv'
        data4 = pd.read_csv(PATH + fname, encoding = DATASET_ENCODING, index_col=0)

        text.extend(data1.text)
        timestamp.extend(data1.timestamp)
        clean_text.extend(data1.clean_text)
        pred_sadness.extend(data0.pred_sadness)
        pred_fear.extend(data1.pred_fear)
        pred_confident.extend(data2.pred_confident)
        pred_analytical.extend(data3.pred_analytical)
        pred_anger.extend(data4.pred_anger)

df = {'text': text,
      'timestamp': timestamp,
      'clean_text': clean_text, 
      'pred_sadness':  pred_sadness,
      'pred_fear' : pred_fear,
      'pred_confident': pred_confident,
      'pred_analytical': pred_analytical,
      'pred_anger': pred_anger
      }

In [ ]:
covid_keys = ['corona', '#covid19','covid', 'covid19', 'covid-19','pandemic','lockdown','stayhome','staysafe','virus', 'wuhan virus', 'curfew']
final_data = pd.DataFrame(df)
print(len(final_data))
final_data2 = final_data[np.sum([final_data.pred_sadness , final_data.pred_fear, final_data.pred_confident, final_data.pred_analytical, final_data.pred_anger], axis=0)==1]
final_data2.reset_index();
print(len(final_data2))
final_data3 = final_data2[final_data2.clean_text.str.lower().str.contains('|'.join(covid_keys))]
final_data3.reset_index();
print(len(final_data3))


datetime = pd.to_datetime(final_data3['timestamp']).to_frame()
datetime.columns=['datetime']
date = datetime['datetime'].dt.date.to_frame()
time = datetime['datetime'].dt.time.to_frame()

date.columns=['date']
time.columns=['time']

final_data3 = pd.concat([date, time, final_data3], axis=1)


In [ ]:
final_data3

In [ ]:
labels = np.array([final_data3.pred_sadness, final_data3.pred_fear, final_data3.pred_confident, final_data3.pred_analytical, final_data3.pred_anger]).astype(int)
out_labels = np.argmax(labels, axis=0)

In [ ]:
final_data3.drop(columns=['pred_sadness', 'pred_fear', 'pred_confident', 'pred_analytical', 'pred_anger'], inplace=True)
final_data3['labels'] = out_labels
final_data3.drop(columns=['timestamp'], inplace=True)

In [ ]:
final_data3.head()

In [ ]:
final_data3.to_csv('/content/drive/My Drive/DL_Project/notebookks/basit/new_data/final_dataset.csv', index=False)

##### Clean New Scrapped Data

In [ ]:
!pip install langdetect

import pandas as pd
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re
import numpy as np


import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

In [ ]:
path = '/content/drive/My Drive/DL_Project/notebookks/basit/new scrapped data/'

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
covid_twitter_data = pd.DataFrame()

for i in np.arange(1, 8):
    data = pd.read_csv(path + "corona_tweets_0"+str(i)+".csv", encoding=DATASET_ENCODING)
    covid_twitter_data = covid_twitter_data.append(data, ignore_index=True)

In [ ]:
english_data = covid_twitter_data[covid_twitter_data.lang.values == 'en']
english_data = english_data.reset_index(drop=True)
english_data.drop(columns=['status_id','user_id','is_quote','display_text_width','favorite_count','retweet_count','hashtags','symbols','lang'], inplace=True)
english_data

In [ ]:
clean_text = []
non_en = []
error = []
index = 0
for item in english_data['text']:
    try:
        index = index +1
        # Delete url in text
        item = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|(\xa0(…)*)))+){1}','',item)
        # Delete @
        item = re.sub('@[a-zA-Z0-9_.-]+','',item)

        try:
            # Detect whether twitter is in English or not 
            if detect(item) == 'en':
                clean_text.append(item)
            else:
                clean_text.append(None)
                non_en.append(index)
        except:
            error.append(index)
            clean_text.append(None)
    except:
        error.append(index)
        clean_text.append(None)

In [ ]:
error_df = pd.DataFrame(error)
non_en_df = pd.DataFrame(non_en)
error_df.to_csv(path + 'error_df.csv',index=False)
non_en_df.to_csv(path + 'non_en_df.csv',index=False)

In [ ]:
clean_text = pd.DataFrame(clean_text)
clean_text.columns=['clean_text']
data = pd.concat([english_data, clean_text], axis=1)
data.to_csv(path + 'scrapped_clean.csv',index=False)
data.head()

In [ ]:
data_strip_none = data[data.clean_text.notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))
covid_keys = ['corona', '#covid19','covid', 'covid19', 'covid-19','pandemic','lockdown','stayhome','staysafe','virus', 'wuhan virus', 'curfew']
final_data = data_strip_none[data_strip_none.clean_text.str.lower().str.contains('|'.join(covid_keys))]
final_data.reset_index();
print(len(final_data))

In [ ]:
final_data.drop_duplicates(keep='first', inplace=True)
final_data = final_data[final_data.clean_text.str.count('\s+').gt(3)]
print(len(final_data))
final_data.to_csv(path + '/final_dataset.csv', index=False)

In [ ]:
final_data.head()

##### Dehydrate Tweets from IEEE Dataset

In [ ]:
#@title Set up Directory { run: "auto"}
import os
from IPython.display import clear_output
from google.colab import drive 
from IPython.display import clear_output
drive.mount('/content/gdrive')
working_directory = 'My Drive/DL_Project/notebookks/basit/covid19-twitter-data/at2/' #@param {type:"string"}
wd="/content/gdrive/"+working_directory
os.chdir(wd)

dirpath = os.getcwd()
print("current directory is : " + dirpath)


In [ ]:
%pip install twarc
%pip install jsonlines
clear_output()

In [ ]:
#Check if TWARC was installed correctly on the Virtual Machine
%pip show twarc
%pip show jsonlines
clear_output()

In [ ]:
#Insert API Keys here { run : "auto"}
from twarc import Twarc

# These keys are received after applying for a twitter developer account

consumer_key = "lvptRYZWgq5PzvJwOAgIc2EWU" #@param {type:"string"}
consumer_secret = "xpB4QsR9patLOhMSKiWTUORSMN6bRYLQjzogWDadsZKQcTwlkO" #@param {type:"string"}
access_token = "1560119378-HNflkzuQUciTlSws91qcGWIU6aT8is0QrETUWwg" #@param {type:"string"}
access_token_secret = "YCrhUpGXCk1UWjIKU9VML5sAgXtg62bpGt2dFCs8SFSk4" #@param {type:"string"}

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

In [ ]:
#Set up Directory { run: "auto"}
final_tweet_ids_filename = "tweet_ids3.csv" #@param {type: "string"}
output_filename = "final_tweets3.csv" #@param {type: "string"}

In [ ]:
import jsonlines, json
# Stores hydrated tweets here as jsonl objects
# Contains one json object per line
output_json_filename = output_filename[:output_filename.index(".")] + ".txt"
ids = []
with open(final_tweet_ids_filename, "r") as ids_file:
    ids = ids_file.read().split()
hydrated_tweets = []
ids_to_hydrate = set(ids)

# Looks at the output file for already hydrated tweets
if os.path.isfile(output_json_filename):
    with jsonlines.open(output_json_filename, "r") as reader:
        for i in reader.iter(type=dict, skip_invalid=True):
            # These tweets have already been hydrated. So remove them from ids_to_hydrate
            hydrated_tweets.append(i)
            ids_to_hydrate.remove(i["id_str"])
print("Total IDs: " + str(len(ids)) + ", IDs to hydrate: " + str(len(ids_to_hydrate)))
print("Hydrated: " + str(len(hydrated_tweets)))



In [ ]:
count = len(hydrated_tweets)
start_index = count # The index from where tweets haven't been saved to the output_json_file
# Stores hydrated tweets to output_json_file every num_save iterations.
num_save  = 1000

# Now, use twarc and start hydrating
for tweet in t.hydrate(ids_to_hydrate):
    hydrated_tweets.append(tweet)
    count += 1
    # If num_save iterations have passed,
    if (count % num_save) == 0:
        # Open the output file
        # NOTE: Even if the code stops during IO, only tweets from the current iteration are lost.
        # Older tweets are preserved as the file is written in append mode.
        with jsonlines.open(output_json_filename, "a") as writer:
            print("Started IO")
            # Now write the tweets from start_index. The other tweets don't have to be written
            # as they were already written in a previous iteration or run.
            for hydrated_tweet in hydrated_tweets[start_index:]:
                writer.write(hydrated_tweet)
            print("Finished IO")
        print("Saved " + str(count) + " hydrated tweets.")
        # Now, since everything has been written. Reset start_index
        start_index = count
# There might be tweets unwritten in the last iteration if the count is not a multiple of num_tweets.
# In that case, just write out the remainder of tweets.
if count != start_index:
    print("Here with start_index", start_index)
    with jsonlines.open(output_json_filename, "a") as writer:
        for hydrated_tweet in hydrated_tweets[start_index:]:
           writer.write(hydrated_tweet)   

In [ ]:
# Convert jsonl to csv
import csv, jsonlines
output_json_filename = output_filename[:output_filename.index(".")] + ".txt"
# These are the column name that are selected to be stored in the csv
keyset = ["created_at", "id", "id_str", "full_text", "source", "truncated", "in_reply_to_status_id",
          "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str", 
          "in_reply_to_screen_name", "user", "coordinates", "place", "quoted_status_id",
          "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status", 
          "quote_count", "reply_count", "retweet_count", "favorite_count", "entities", 
          "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level", 
          "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright", 
          "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range",
          "quoted_status_permalink"]
hydrated_tweets = []
# Reads the current tweets
with jsonlines.open(output_json_filename, "r") as reader:
    for i in reader.iter(type=dict, skip_invalid=True):
        hydrated_tweets.append(i)
# Writes them out
with  open(output_filename, "w+") as output_file:
    d = csv.DictWriter(output_file, keyset)
    d.writeheader()
    d.writerows(hydrated_tweets)

##### Cleaning Rehydrated Data

In [ ]:
!pip install langdetect

import pandas as pd
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re
import numpy as np


import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
path = '/content/drive/My Drive/DL_Project/notebookks/basit/ieee data/'
covid_twitter_data = pd.read_csv(path + "final_tweets.csv", encoding=DATASET_ENCODING)
covid_twitter_data

In [ ]:
import numpy as np
covid_twitter_data.place.replace('True',None, inplace=True)
covid_twitter_data.place.replace('False',None, inplace=True)

def ll(x):
    if x != x:
      #print(x)
      y=None
    else:
      y = literal_eval(x)
      y = y['country']
    return y

covid_twitter_data["country"] = covid_twitter_data.place.apply(lambda x: ll(x))

In [ ]:
english_data = covid_twitter_data[covid_twitter_data.lang.values == 'en']
english_data = english_data.reset_index(drop=True)
english_data.drop(columns=["source","id", "id_str", "truncated", "in_reply_to_status_id", "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str",  "in_reply_to_screen_name", "user",  "quoted_status_id", "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status",  "quote_count", "reply_count", "retweet_count", "favorite_count", "entities",  "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level",  "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright",  "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range", "quoted_status_permalink"], inplace=True)
english_data.columns=['timestamp','text','coordinates','place','country']
english_data

In [ ]:
datetime = pd.to_datetime(english_data['timestamp']).to_frame()
datetime.columns=['datetime']
date = datetime['datetime'].dt.date.to_frame()
time = datetime['datetime'].dt.time.to_frame()

date.columns=['date']
time.columns=['time']

english_data = pd.concat([date, time, english_data], axis=1)
english_data.drop(columns=['timestamp'], inplace=True)

In [ ]:
clean_text = []
non_en = []
error = []
index = 0
for item in english_data['text']:
    try:
        index = index +1
        # Delete url in text
        item = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|(\xa0(…)*)))+){1}','',item)
        # Delete @
        item = re.sub('@[a-zA-Z0-9_.-]+','',item)

        try:
            # Detect whether twitter is in English or not 
            if detect(item) == 'en':
                clean_text.append(item)
            else:
                clean_text.append(None)
                non_en.append(index)
        except:
            error.append(index)
            clean_text.append(None)
    except:
        error.append(index)
        clean_text.append(None)

In [ ]:
error_df = pd.DataFrame(error)
non_en_df = pd.DataFrame(non_en)
error_df.to_csv(path + 'error_df.csv',index=False)
non_en_df.to_csv(path + 'non_en_df.csv',index=False)

In [ ]:
clean_text = pd.DataFrame(clean_text)
clean_text.columns=['clean_text']
data = pd.concat([english_data, clean_text], axis=1)
data.to_csv(path + 'scrapped_clean.csv',index=False)
data.head()

In [ ]:
data_strip_none = data[data.clean_text.notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))
covid_keys = ['corona', '#covid19','covid', 'covid19', 'covid-19','pandemic','lockdown','stayhome','staysafe','virus', 'wuhan virus', 'curfew']
final_data = data_strip_none[data_strip_none.clean_text.str.lower().str.contains('|'.join(covid_keys))]
final_data.reset_index();
print(len(final_data))

In [ ]:
final_data.drop_duplicates(keep='first', inplace=True)
final_data = final_data[final_data.clean_text.str.count('\s+').gt(3)]
print(len(final_data))
final_data.to_csv(path + '/final_dataset.csv', index=False)

In [ ]:
final_data.head()

In [ ]:
final_data.country.replace(None,np.nan, inplace=True)

In [ ]:
final_data

In [ ]:
target_cnt = Counter(final_data.country)
target_cnts = {k: v for k, v in sorted(target_cnt.items(), key=lambda item: item[1], reverse=True)}

target_cnts = {k: target_cnts[k] for k in list(target_cnts)[:20]}

plt.figure(figsize=(10,5))
plt.bar(target_cnts.keys(),target_cnts.values())
plt.title("Dataset labels distribuition")
plt.gcf().autofmt_xdate(rotation=90)

##### Cleaning Rehydrated GEO tWEETS

In [ ]:
!pip install langdetect

import pandas as pd
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re
import numpy as np


import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
from ast import literal_eval
import ast

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
path = '/content/drive/My Drive/DL_Project/notebookks/basit/ieee geo data/'
covid_twitter_data = pd.read_csv(path + "final_tweets.csv", encoding=DATASET_ENCODING)
len(covid_twitter_data)

In [ ]:
import numpy as np
covid_twitter_data.place.replace('True',None, inplace=True)
covid_twitter_data.place.replace('False',None, inplace=True)

def ll(x):
    if x != x:
      #print(x)
      y=None
    else:
      y = literal_eval(x)
      y = y['country']
    return y

covid_twitter_data["country"] = covid_twitter_data.place.apply(lambda x: ll(x))

In [ ]:
english_data = covid_twitter_data[covid_twitter_data.lang.values == 'en']
english_data = english_data.reset_index(drop=True)
english_data.drop(columns=["source","id", "id_str", "truncated", "in_reply_to_status_id", "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str",  "in_reply_to_screen_name", "user",  "quoted_status_id", "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status",  "quote_count", "reply_count", "retweet_count", "favorite_count", "entities",  "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level",  "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright",  "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range", "quoted_status_permalink"], inplace=True)
english_data.columns=['timestamp','text','coordinates','place','country']
english_data

In [ ]:
datetime = pd.to_datetime(english_data['timestamp']).to_frame()
datetime.columns=['datetime']
date = datetime['datetime'].dt.date.to_frame()
time = datetime['datetime'].dt.time.to_frame()

date.columns=['date']
time.columns=['time']

english_data = pd.concat([date, time, english_data], axis=1)
english_data.drop(columns=['timestamp'], inplace=True)

In [ ]:
clean_text = []
non_en = []
error = []
index = 0
for item in english_data['text']:
    try:
        index = index +1
        # Delete url in text
        item = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|(\xa0(…)*)))+){1}','',item)
        # Delete @
        item = re.sub('@[a-zA-Z0-9_.-]+','',item)

        try:
            # Detect whether twitter is in English or not 
            if detect(item) == 'en':
                clean_text.append(item)
            else:
                clean_text.append(None)
                non_en.append(index)
        except:
            error.append(index)
            clean_text.append(None)
    except:
        error.append(index)
        clean_text.append(None)

In [ ]:
path = '/content/drive/My Drive/DL_Project/notebookks/basit/ieee geo data/'
error_df = pd.DataFrame(error)
non_en_df = pd.DataFrame(non_en)
error_df.to_csv(path + 'error_df.csv',index=False)
non_en_df.to_csv(path + 'non_en_df.csv',index=False)

In [ ]:
clean_text = pd.DataFrame(clean_text)
clean_text.columns=['clean_text']
data = pd.concat([english_data, clean_text], axis=1)
data.to_csv(path + 'scrapped_clean.csv',index=False)
data.head()

In [ ]:
data_strip_none = data[data.clean_text.notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))
covid_keys = ['corona', '#covid19','covid', 'covid19', 'covid-19','pandemic','lockdown','stayhome','staysafe','virus', 'wuhan virus', 'curfew']
final_data = data_strip_none[data_strip_none.clean_text.str.lower().str.contains('|'.join(covid_keys))]
final_data.reset_index();
print(len(final_data))

In [ ]:
final_data.drop_duplicates(keep='first', inplace=True)
final_data = final_data[final_data.clean_text.str.count('\s+').gt(3)]
print(len(final_data))
final_data.to_csv(path + '/final_dataset.csv', index=False)

In [ ]:
final_data.head()

In [ ]:
target_cnt = Counter(final_data.country)
target_cnts = {k: v for k, v in sorted(target_cnt.items(), key=lambda item: item[1], reverse=True)}

target_cnts = {k: target_cnts[k] for k in list(target_cnts)[:20]}

plt.figure(figsize=(10,5))
plt.bar(target_cnts.keys(),target_cnts.values())
plt.title("Dataset labels distribuition")
plt.gcf().autofmt_xdate(rotation=90)

##### Panencia LAB

In [ ]:
!pip install langdetect

import pandas as pd
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import re
import numpy as np


import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
from ast import literal_eval
import ast

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
covid_twitter_data = pd.DataFrame()
covid_twitter_data1 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/covid19-twitter-data/at1/final_tweets.csv', encoding=DATASET_ENCODING)
covid_twitter_data2 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/covid19-twitter-data/at2/final_tweets.csv', encoding=DATASET_ENCODING)



covid_twitter_data = covid_twitter_data.append(covid_twitter_data1, ignore_index=True)
covid_twitter_data = covid_twitter_data.append(covid_twitter_data2, ignore_index=True)

covid_twitter_data.to_csv('/content/drive/My Drive/DL_Project/notebookks/basit/covid19-twitter-data/final_tweets.csv', index=False)

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
path = '/content/drive/My Drive/DL_Project/notebookks/basit/covid19-twitter-data/'
covid_twitter_data = pd.read_csv(path + "final_tweets.csv", encoding=DATASET_ENCODING)
len(covid_twitter_data)

In [ ]:
import numpy as np
covid_twitter_data.place.replace('True',None, inplace=True)
covid_twitter_data.place.replace('False',None, inplace=True)

def ll(x):
    if x != x:
      #print(x)
      y=None
    else:
      y = literal_eval(x)
      y = y['country']
    return y

covid_twitter_data["country"] = covid_twitter_data.place.apply(lambda x: ll(x))

In [ ]:
english_data = covid_twitter_data[covid_twitter_data.lang.values == 'en']
english_data = english_data.reset_index(drop=True)
english_data.drop(columns=["source","id", "id_str", "truncated", "in_reply_to_status_id", "in_reply_to_status_id_str", "in_reply_to_user_id", "in_reply_to_user_id_str",  "in_reply_to_screen_name", "user",  "quoted_status_id", "quoted_status_id_str", "is_quote_status", "quoted_status", "retweeted_status",  "quote_count", "reply_count", "retweet_count", "favorite_count", "entities",  "extended_entities", "favorited", "retweeted", "possibly_sensitive", "filter_level",  "lang", "matching_rules", "current_user_retweet", "scopes", "withheld_copyright",  "withheld_in_countries", "withheld_scope", "geo", "contributors", "display_text_range", "quoted_status_permalink"], inplace=True)
english_data.columns=['timestamp','text','coordinates','place','country']
english_data

In [ ]:
datetime = pd.to_datetime(english_data['timestamp']).to_frame()
datetime.columns=['datetime']
date = datetime['datetime'].dt.date.to_frame()
time = datetime['datetime'].dt.time.to_frame()

date.columns=['date']
time.columns=['time']

english_data = pd.concat([date, time, english_data], axis=1)
english_data.drop(columns=['timestamp'], inplace=True)

In [ ]:
clean_text = []
non_en = []
error = []
index = 0
for item in english_data['text']:
    try:
        index = index +1
        # Delete url in text
        item = re.sub('(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]|(\xa0(…)*)))+){1}','',item)
        # Delete @
        item = re.sub('@[a-zA-Z0-9_.-]+','',item)

        try:
            # Detect whether twitter is in English or not 
            if detect(item) == 'en':
                clean_text.append(item)
            else:
                clean_text.append(None)
                non_en.append(index)
        except:
            error.append(index)
            clean_text.append(None)
    except:
        error.append(index)
        clean_text.append(None)

In [ ]:
path = '/content/drive/My Drive/DL_Project/notebookks/basit/covid19-twitter-data/'
error_df = pd.DataFrame(error)
non_en_df = pd.DataFrame(non_en)
error_df.to_csv(path + 'error_df.csv',index=False)
non_en_df.to_csv(path + 'non_en_df.csv',index=False)

In [ ]:
clean_text = pd.DataFrame(clean_text)
clean_text.columns=['clean_text']
data = pd.concat([english_data, clean_text], axis=1)
data.to_csv(path + 'scrapped_clean.csv',index=False)
data.head()

In [ ]:
data_strip_none = data[data.clean_text.notnull()]
data_strip_none = data_strip_none.set_index(pd.Index(range(0,len(data_strip_none))))
covid_keys = ['corona', '#covid19','covid', 'covid19', 'covid-19','pandemic','lockdown','stayhome','staysafe','virus', 'wuhan virus', 'curfew']
final_data = data_strip_none[data_strip_none.clean_text.str.lower().str.contains('|'.join(covid_keys))]
final_data.reset_index();
print(len(final_data))

In [ ]:
final_data.drop_duplicates(keep='first', inplace=True)
final_data = final_data[final_data.clean_text.str.count('\s+').gt(3)]
print(len(final_data))
final_data.to_csv(path + '/final_dataset.csv', index=False)

In [ ]:
final_data.head()

In [ ]:
target_cnt = Counter(final_data.country)
target_cnts = {k: v for k, v in sorted(target_cnt.items(), key=lambda item: item[1], reverse=True)}

target_cnts = {k: target_cnts[k] for k in list(target_cnts)[:20]}

plt.figure(figsize=(10,5))
plt.bar(target_cnts.keys(),target_cnts.values())
plt.title("Dataset labels distribuition")
plt.gcf().autofmt_xdate(rotation=90)

##### Combine Rehydrated and Scrapped Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
covid_twitter_data = pd.DataFrame()
covid_twitter_data1 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/new scrapped data/final_dataset.csv', encoding=DATASET_ENCODING)
covid_twitter_data2 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/ieee data/final_dataset.csv', encoding=DATASET_ENCODING)
covid_twitter_data3 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/ieee geo data/final_dataset.csv', encoding=DATASET_ENCODING)
covid_twitter_data4 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/covid19-twitter-data/final_dataset.csv', encoding=DATASET_ENCODING)
covid_twitter_data5 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/new_data/final_dataset.csv', encoding=DATASET_ENCODING)

In [ ]:
covid_twitter_data1["coordinates"] = np.nan
covid_twitter_data1["place"] = np.nan
covid_twitter_data1["country"] = np.nan

covid_twitter_data1 = covid_twitter_data1[covid_twitter_data2.columns]

In [ ]:
covid_twitter_data5["coordinates"] = np.nan
covid_twitter_data5["place"] = np.nan
covid_twitter_data5["country"] = np.nan

covid_twitter_data5 = covid_twitter_data5[['date','time','text','coordinates','place','country','clean_text','labels']]

In [ ]:
covid_twitter_data = covid_twitter_data.append(covid_twitter_data1, ignore_index=True)
covid_twitter_data = covid_twitter_data.append(covid_twitter_data2, ignore_index=True)
covid_twitter_data = covid_twitter_data.append(covid_twitter_data3, ignore_index=True)
covid_twitter_data = covid_twitter_data.append(covid_twitter_data4, ignore_index=True)

In [ ]:
print(len(covid_twitter_data))
covid_twitter_data.drop_duplicates(keep='first', inplace=True)
print(len(covid_twitter_data))

In [ ]:
covid_twitter_data.to_csv('/content/drive/My Drive/DL_Project/notebookks/basit/final_unseen_dataset.csv', index=False)

In [ ]:
covid_twitter_data5.drop(columns = ['labels'], inplace=True)
covid_twitter_data = covid_twitter_data.append(covid_twitter_data5, ignore_index=True)
covid_twitter_data.drop_duplicates(keep='first', inplace=True)
print(len(covid_twitter_data))
covid_twitter_data.to_csv('/content/drive/My Drive/DL_Project/notebookks/basit/final_w2v_dataset.csv', index=False)

##### Draw Figures (Freq and Word Clouds)

In [ ]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
% matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

import gensim

import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools
import PIL

!git clone https://github.com/amueller/word_cloud.git
!cd word_cloud
!pip install .

nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

import datetime
import matplotlib.dates as dates 

In [ ]:
PATH = '/content/drive/My Drive/DL_Project/notebookks/basit/'
filename = 'final_dataset.csv'
df = pd.read_csv(PATH + filename)

In [ ]:
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9\']+"
def preprocess(text, stem=False):
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            tokens.append(token)
    text = " ".join(tokens)
    return text
df.clean_text = df.clean_text.apply(lambda x: preprocess(x))

In [ ]:
text = " ".join(cl for cl in df.clean_text)
words = text.split(' ')

counts_no_urls = collections.Counter(words)


clean_tweets_no_urls = pd.DataFrame(counts_no_urls.most_common(20), columns=['words', 'count'])
fig, ax = plt.subplots(figsize=(10, 5))
clean_tweets_no_urls.sample(frac=1).sample(frac=1).plot.bar(x='words', y='count', ax=ax, width = 0.9)
plt.ylabel('Frequency')
ax.set_title("Most common words")
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/freq plots/freq_plot'+ '.tiff', bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/freq plots/freq_plot'+ '.pdf', bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/freq plots/freq_plot'+ '.png', bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.show()
print ("There are {} words in the combination of all review.".format(len(text)))

In [ ]:
wordcloud = WordCloud(max_font_size=50, max_words=220, background_color="black").generate(text)
plt.figure(figsize = (10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/word clouds/word_cloud.tiff',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/word clouds/word_cloud.pdf',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/word clouds/word_cloud.png',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.show()

In [ ]:
e = ['sadness','fear','confident','analytical','anger']
for i in np.arange(0, 5):
    dfi = df[df.labels ==i]

    text = " ".join(cl for cl in dfi.clean_text)
    words = text.split(' ')

    counts_no_urls = collections.Counter(words)


    clean_tweets_no_urls = pd.DataFrame(counts_no_urls.most_common(20), columns=['words', 'count'])
    fig, ax = plt.subplots(figsize=(10, 5))
    clean_tweets_no_urls.sample(frac=1).plot.bar(x='words', y='count', ax=ax,width=0.9)
    plt.ylabel('Frequency')
    ax.set_title("Most Frequent Words (" + e[i] + ")")
    plt.savefig('./drive/My Drive/DL_Project/notebookks/results/freq plots/freq_plot_'+ str(e[i])+ '.tiff', bbox_inches='tight',pad_inches = 0.05, dpi = 400)
    plt.savefig('./drive/My Drive/DL_Project/notebookks/results/freq plots/freq_plot_'+ str(e[i])+ '.pdf', bbox_inches='tight',pad_inches = 0.05, dpi = 400)
    plt.savefig('./drive/My Drive/DL_Project/notebookks/results/freq plots/freq_plot_'+ str(e[i])+ '.png', bbox_inches='tight',pad_inches = 0.05, dpi = 400)
    plt.show()
    print ("There are {} words in the combination of all review.".format(len(text)))
    wordcloud = WordCloud(max_font_size=50, max_words=200, background_color="black").generate(text)
    plt.figure(figsize = (10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.savefig('./drive/My Drive/DL_Project/notebookks/results/word clouds/word_cloud_'+ str(e[i])+ '.tiff',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
    plt.savefig('./drive/My Drive/DL_Project/notebookks/results/word clouds/word_cloud_'+ str(e[i])+ '.pdf',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
    plt.savefig('./drive/My Drive/DL_Project/notebookks/results/word clouds/word_cloud_'+ str(e[i])+ '.png',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
    plt.axis("off")
    #plt.show()
    

In [ ]:
e = ['sadness','fear','confident','analytical','anger']
target_cnt = Counter(df.labels)
plt.figure(figsize=(8,5))
plt.bar(np.array(e)[np.array(list(target_cnt.keys()))], target_cnt.values())
plt.ylabel('Number of Tweets')
plt.xlabel('Sentiments Class')
plt.title("Dataset labels distribuition")
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_labels'+ '.tiff',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_labels'+ '.pdf',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_labels'+ '.png',bbox_inches='tight',pad_inches = 0.05, dpi = 400)

##### Generate Figure (Dataset Distribution Date)

In [ ]:
import numpy as np
import pandas as pd
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
% matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

import gensim

import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools
import PIL

!git clone https://github.com/amueller/word_cloud.git
!cd word_cloud
!pip install .

nltk.download('stopwords')
stop_words = stopwords.words('english')
stemmer = SnowballStemmer('english')

import datetime
import matplotlib.dates as dates 

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
PATH = '/content/drive/My Drive/DL_Project/notebookks/basit/'
filename = 'final_w2v_dataset.csv'
df = pd.read_csv(PATH + filename, encoding=DATASET_ENCODING)

In [ ]:
df[df.date == 'https://t.co/vekDÃ\x83Â¢Ã\x82Â\x80Ã\x82Â¦'] = ''
df[df.date == 'https://t.cÃ\x83Â¢Ã\x82Â\x80Ã\x82Â¦'] = ''
df[df.date == 'Ã\x83Â¢Ã\x82Â\x80Ã\x82Â¦'] = ''

In [ ]:
## Date Distribution wit dates

target_cnts = Counter(df.date)
target_cnts = {k: v for k, v in sorted(target_cnts.items(), key=lambda item: item[0])}
target_cnts = {k: target_cnts[k] for k in list(target_cnts)[1:]}
dates1 = list(target_cnts.keys())
converted_dates = list(map(datetime.datetime.strptime, dates1, len(dates1)*['%Y-%m-%d']))
formatter = dates.DateFormatter('%Y-%m-%d')

plt.figure(figsize=(45,10))
plt.bar(converted_dates,target_cnts.values())
plt.ylabel('Number of Tweets')
plt.xlabel('Dates')
ax = plt.gcf().axes[0] 
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 1))
ax.xaxis.set_major_formatter(formatter)
plt.gcf().autofmt_xdate(rotation=90)
ax.set_xlim([datetime.date(2020, 2, 19), datetime.date(2020, 7, 6)])
plt.title("Tweets Distribution with Dates")


plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_dates'+ '.png',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_dates'+ '.pdf',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_dates'+ '.tiff',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.show()

In [ ]:
target_cnts

 ##### Generate Figure (Dataset Distribution Country)

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
final_data = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/final_w2v_dataset.csv', encoding=DATASET_ENCODING)

In [ ]:
final_data.dropna(subset=['place'], inplace=True)

target_cnts = Counter(final_data.country)
target_cnts = {k: v for k, v in sorted(target_cnts.items(), key=lambda item: item[1], reverse=True)}

target_cnts = {k: target_cnts[k] for k in list(target_cnts)[:20]}

plt.figure(figsize=(10,5))
plt.bar(target_cnts.keys(),target_cnts.values(), width=0.9)
plt.xlabel('Countries')
plt.ylabel('Number of Tweets')
plt.title("Tweets Distribution with Countries")
plt.gcf().autofmt_xdate(rotation=90)

plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_cont'+ '.png',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_cont'+ '.pdf',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results/dataset_distribution_cont'+ '.tiff',bbox_inches='tight',pad_inches = 0.05, dpi = 400)

plt.show()

 ##### Generate Figure (Datewise Plot)

In [ ]:
DATASET_ENCODING = "ISO-8859-1"
covid_twitter_data = pd.DataFrame()
covid_twitter_data1 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/basit/final_dataset.csv', encoding=DATASET_ENCODING)
covid_twitter_data2 = pd.read_csv('/content/drive/My Drive/DL_Project/notebookks/results/Copy of Predictions.csv', encoding=DATASET_ENCODING)

In [ ]:
covid_twitter_data2 = covid_twitter_data2.rename(columns = {'clean_txt': 'clean_text'})

In [ ]:
covid_twitter_data1["coordinates"] = np.nan
covid_twitter_data1["place"] = np.nan
covid_twitter_data1["country"] = np.nan
covid_twitter_data1["emotions"] = np.nan

covid_twitter_data1 = covid_twitter_data1[covid_twitter_data2.columns]

covid_twitter_data = covid_twitter_data.append(covid_twitter_data1)
covid_twitter_data = covid_twitter_data.append(covid_twitter_data2)

In [ ]:
covid_twitter_data[covid_twitter_data.date == 'https://t.co/vekDÃ\x83Â¢Ã\x82Â\x80Ã\x82Â¦'] = ''
covid_twitter_data[covid_twitter_data.date == 'https://t.cÃ\x83Â¢Ã\x82Â\x80Ã\x82Â¦'] = ''
covid_twitter_data[covid_twitter_data.date == 'Ã\x83Â¢Ã\x82Â\x80Ã\x82Â¦'] = ''
covid_twitter_data[covid_twitter_data.date == 'https://t.co/vekDÃ\x83Â\x83Ã\x82Â¢Ã\x83Â\x82Ã\x82Â\x80Ã\x83Â\x82Ã\x82Â¦'] = ''
covid_twitter_data[covid_twitter_data.date == 'https://t.cÃ\x83Â\x83Ã\x82Â¢Ã\x83Â\x82Ã\x82Â\x80Ã\x83Â\x82Ã\x82Â¦'] = ''
covid_twitter_data[covid_twitter_data.date == 'Ã\x83Â\x83Ã\x82Â¢Ã\x83Â\x82Ã\x82Â\x80Ã\x83Â\x82Ã\x82Â¦'] = ''

covid_twitter_data.reset_index()

In [ ]:
# DATASET_ENCODING = "ISO-8859-1"
# PATH = '/content/drive/My Drive/DL_Project/notebookks/basit/'
# filename = 'final_w2v_dataset.csv'
# df = pd.read_csv(PATH + filename, encoding=DATASET_ENCODING)
# df

In [ ]:
dates1 = np.unique(covid_twitter_data.date)
dates1 = dates1[1:]
emotions = []
for d in dates1:
    covid_date_data = (covid_twitter_data[covid_twitter_data.date==d]).reset_index(drop=True)
    target_cnts = Counter(covid_date_data.labels)
    target_cnts = {k: v for k, v in sorted(target_cnts.items(), key=lambda item: item[0])}
    emotions.append([x[1] for x in target_cnts.items()])
    

emotions = np.array(emotions)
emotion = emotions/np.sum(emotions, axis=1).reshape(135,1)

em = []
dates2 = []
for d in np.arange(0, 131, 5):
    st = np.min(dates1[d:d+5])
    dates2.append(st)

    emoti = np.sum(emotion[d:d+5], axis=0)
    em.append(emoti)

e = ['sadness','fear','confident','analytical','anger']
converted_dates = list(map(datetime.datetime.strptime, dates2, len(dates2)*['%Y-%m-%d']))
formatter = dates.DateFormatter('%Y-%m-%d')


f = plt.figure(figsize=(15, 5))
plt.plot(converted_dates, eem,'o-',linewidth=3, markersize=8)
plt.legend(e)
plt.xlabel('dates')
plt.ylabel('Number of Tweets')
ax = plt.gcf().axes[0] 
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start, end, 5))
ax.xaxis.set_major_formatter(formatter)
plt.gcf().autofmt_xdate(rotation=90)


plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/datewise_emotion'+ '.png',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/datewise_emotion'+ '.pdf',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/datewise_emotion'+ '.tiff',bbox_inches='tight',pad_inches = 0.05, dpi = 400)

plt.show()

 ##### Generate Figure (Countrywise Plot)

In [ ]:
covid_twitter_data.dropna(subset=['country'], inplace=True)
covid_twitter_data11 = covid_twitter_data


target_cnts = Counter(covid_twitter_data.country)
target_cnts = {k: v for k, v in sorted(target_cnts.items(), key=lambda item: item[1], reverse=True)}
target_cnts = {k: target_cnts[k] for k in list(target_cnts)[:5]}
countries = list(target_cnts.keys())
countries.append('Pakistan')

for c in countries:
    covid_country_data = covid_twitter_data[covid_twitter_data.country == c]
    dates1 = np.unique(covid_country_data.date)
    dates1 = dates1[1:]

    emotions = []
    for d in dates1:
        covid_date_data = (covid_country_data[covid_country_data.date==d]).reset_index(drop=True)
        target_cnts = Counter(covid_date_data.labels)
        target_cnts = {k: v for k, v in sorted(target_cnts.items(), key=lambda item: item[0])}
        aa = np.zeros((5))
        for k in target_cnts.items():
            aa[k[0]] = k[1]
        emotions.append(aa)

    emotions = np.array(emotions)
    emotion = emotions/np.sum(emotions, axis=1).reshape(len(dates1),1)

    em = []
    dates2 = []
    for d in np.arange(0, len(dates1)-2, 5):
        st = np.min(dates1[d:d+5])
        dates2.append(st)

        emoti = np.sum(emotion[d:d+5], axis=0)
        em.append(emoti)
    eem = np.array(em)

    e = ['sadness','fear','confident','analytical','anger']
    converted_dates = list(map(datetime.datetime.strptime, dates2, len(dates2)*['%Y-%m-%d']))
    formatter = dates.DateFormatter('%Y-%m-%d')


    f = plt.figure(figsize=(15, 5))
    p = plt.plot(converted_dates, eem,'o-',linewidth=3, markersize=8)
    plt.legend(e)
    plt.xlabel('dates')
    plt.ylabel('Number of Tweets')
    plt.title('Sentiment Analysis of ' + c  + ' with Dates')
    ax = plt.gcf().axes[0] 
    start, end = ax.get_xlim()
    ax.xaxis.set_ticks(np.arange(start, end, 5))
    ax.xaxis.set_major_formatter(formatter)
    plt.gcf().autofmt_xdate(rotation=90)


    plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/datewise_country_emotion_'+ c +  '.png',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
    plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/datewise_country_emotion_'+ '.pdf',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
    plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/datewise_country_emotion_'+ '.tiff',bbox_inches='tight',pad_inches = 0.05, dpi = 400)

    plt.show()

In [ ]:
covid_twitter_data.dropna(subset=['country'], inplace=True)
covid_twitter_data11 = covid_twitter_data


target_cnts = Counter(covid_twitter_data.country)
target_cnts = {k: v for k, v in sorted(target_cnts.items(), key=lambda item: item[1], reverse=True)}
target_cnts = {k: target_cnts[k] for k in list(target_cnts)[:13]}
countries = list(target_cnts.keys())

emotions = []
for c in countries:
    covid_country_data = covid_twitter_data[covid_twitter_data.country == c]
    
    target_cnts = Counter(covid_country_data.labels)
    target_cnts = {k: v for k, v in sorted(target_cnts.items(), key=lambda item: item[0])}
    aa = np.zeros((5))
    for k in target_cnts.items():
        aa[k[0]] = k[1]
    
    emotions.append(aa)
emotion = emotions/np.sum(emotions, axis=1).reshape(13,1)
    
cc = np.array(np.arange(1, 27, 2))

plt.figure(figsize = (15, 7))
ax = plt.subplot(111)
ax.bar(cc    , emotion[:,0], width=0.3, color="#1f77b4",align='center')
ax.bar(cc-0.3, emotion[:,1], width=0.3, color='#ff7f0e', align='center')
ax.bar(cc-0.6, emotion[:,2], width=0.3, color='#2ca02c', align='center')
ax.bar(cc+0.3, emotion[:,3], width=0.3, color='#d62728', align='center')
ax.bar(cc+0.6, emotion[:,4], width=0.3, color='#9467bd', align='center')
plt.legend(e)
ax = plt.gcf().axes[0] 
start, end = ax.get_xlim()
ax.xaxis.set_ticks(np.arange(start+2.125*0.95, end, 2))
plt.gcf().autofmt_xdate(rotation=90)
ax.set_xticklabels(countries)

plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/country_emotion'+  '.png',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/country_emotion'+ '.pdf',bbox_inches='tight',pad_inches = 0.05, dpi = 400)
plt.savefig('./drive/My Drive/DL_Project/notebookks/results2/country_emotion'+ '.tiff',bbox_inches='tight',pad_inches = 0.05, dpi = 400)


plt.show()